In [83]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **1- Importing Required Libraries**

In [3]:
import torch
from tqdm.notebook import tqdm

# **2- Load Data**

In [24]:
book_df = pd.read_csv('/root/.cache/kagglehub/datasets/mohamedbakhet/amazon-books-reviews/versions/1/books_data.csv')

book_df.head()

,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
4,"Nation Dance: Religion, Identity and Cultural ...",NaN,['Edward Long'],NaN,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,2003-03-01,http://books.google.nl/books?id=399SPgAACAAJ&d...,NaN,NaN


In [25]:
book_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212404 entries, 0 to 212403
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Title          212403 non-null  object 
 1   description    143962 non-null  object 
 2   authors        180991 non-null  object 
 3   image          160329 non-null  object 
 4   previewLink    188568 non-null  object 
 5   publisher      136518 non-null  object 
 6   publishedDate  187099 non-null  object 
 7   infoLink       188568 non-null  object 
 8   categories     171205 non-null  object 
 9   ratingsCount   49752 non-null   float64
dtypes: float64(1), object(9)
memory usage: 16.2+ MB


In [26]:
missing_values_count = book_df.isnull().sum()
missing_values_count

,0
Title,1
description,68442
authors,31413
image,52075
previewLink,23836
publisher,75886
publishedDate,25305
infoLink,23836
categories,41199
ratingsCount,162652


In [27]:
book_df = book_df.drop(columns = ['description', 'image', 'previewLink', 'publisher', 'publishedDate', 'infoLink', 'ratingsCount'])
book_df = book_df.rename(columns = {'Title':'title'})
book_df = book_df.dropna()
book_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 165744 entries, 0 to 212402
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   title       165744 non-null  object
 1   authors     165744 non-null  object
 2   categories  165744 non-null  object
dtypes: object(3)
memory usage: 5.1+ MB


In [8]:
ratings_df = pd.read_csv('/root/.cache/kagglehub/datasets/mohamedbakhet/amazon-books-reviews/versions/1/Books_rating.csv')

ratings_df.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...


In [9]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Id                  object 
 1   Title               object 
 2   Price               float64
 3   User_id             object 
 4   profileName         object 
 5   review/helpfulness  object 
 6   review/score        float64
 7   review/time         int64  
 8   review/summary      object 
 9   review/text         object 
dtypes: float64(2), int64(1), object(7)
memory usage: 228.9+ MB


In [10]:
missing_values_count = ratings_df.isnull().sum()
missing_values_count

,0
Id,0
Title,208
Price,2518829
User_id,561787
profileName,561905
review/helpfulness,0
review/score,0
review/time,0
review/summary,407
review/text,8


In [11]:
ratings_df = ratings_df.drop(columns = ['Id','Price','profileName', 'review/helpfulness', 'review/time', 'review/summary', 'review/text'])
ratings_df = ratings_df.rename(columns = {'Title':'title', 'User_id':'userId', 'review/score':'rating'})
ratings_df = ratings_df.dropna()
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2438018 entries, 0 to 2999998
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   title   object 
 1   userId  object 
 2   rating  float64
dtypes: float64(1), object(2)
memory usage: 74.4+ MB


In [12]:
print('The dimensions of book dataframe are:', book_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of book dataframe are: (171205, 2) 
The dimensions of ratings dataframe are: (2438018, 3)


In [13]:
ratings_df.head(2)

,title,userId,rating
0,Its Only Art If Its Well Hung!,AVCGYZL8FQQTD,4.0
1,Dr. Seuss: American Icon,A30TK6U7DNS82R,5.0


# **3- Create a mapping of user and item indices**

In [14]:
user_ids = ratings_df['userId'].unique()
item_ids = ratings_df['title'].unique()
user_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
item_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
index_to_title = {idx: title for title, idx in item_to_index.items()}

In [15]:
user_indices = torch.tensor([user_to_index[user_id] for user_id in ratings_df['userId']], dtype=torch.long)
item_indices = torch.tensor([item_to_index[item_id] for item_id in ratings_df['title']], dtype=torch.long)
ratings = torch.tensor(ratings_df['rating'].values, dtype=torch.float32)

# **4- Create a DataLoader**

In [16]:
dataset = torch.utils.data.TensorDataset(user_indices, item_indices, ratings)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

# **5- Collaborative filtering**

In [17]:
class CollaborativeFilteringModel(torch.nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, hidden_dim):
        super().__init__()
        self.user_embedding = torch.nn.Embedding(num_users, embedding_dim)
        self.item_embedding = torch.nn.Embedding(num_items, embedding_dim)
        self.hidden_layer = torch.nn.Linear(embedding_dim * 2, hidden_dim)
        self.relu = torch.nn.ReLU()
        self.output_layer = torch.nn.Linear(hidden_dim, 1)

    def forward(self, user_indices, item_indices):
        user_embedded = self.user_embedding(user_indices)
        item_embedded = self.item_embedding(item_indices)
        concatenated = torch.cat([user_embedded, item_embedded], dim=1)
        hidden_output = self.relu(self.hidden_layer(concatenated))
        output = self.output_layer(hidden_output)
        return output

    def get_similar_titles(self, input_title_index, top_k=100):
        device = self.item_embedding.weight.device  # Get the device of the embeddings

        # Move the input title index to the same device as the model
        input_title_index = torch.tensor([input_title_index], device=device)

        # Get the embedding for the input title
        input_title_embedding = self.item_embedding(input_title_index)

        # Get embeddings for all titles
        all_title_embeddings = self.item_embedding.weight

        # Calculate cosine similarity
        similarities = F.cosine_similarity(input_title_embedding, all_title_embeddings)

        # Get indices of top-k similar titles
        #argsort returns the indices that sort a tensor along a given dimension in ascending order(default) by value.
        similar_title_indices = torch.argsort(similarities, descending=True)[:top_k]

        # Convert indices to a list of titles
        similar_titles = [index_to_title[idx.item()] for idx in similar_title_indices]
        #we are using item() to get scalar value instead of tensor which can be used as an key index for dictionary
        return similar_titles



In [18]:
num_users = len(user_ids)
num_items = len(item_ids)
embedding_dim = 100
model = CollaborativeFilteringModel(num_users, num_items, embedding_dim,32)

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# **6- Training the model**

In [20]:
epochs = 1
for epoch in tqdm(range(epochs),desc="Training Collaborative Model"):
    model.train()
    total_loss = 0.0
    for user_batch, item_batch, rating_batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{epochs}"):
        user_batch, item_batch, rating_batch = user_batch.to(device), item_batch.to(device), rating_batch.to(device)
        #user_batch = user_batch.long()
        #item_batch = item_batch.long()
        optimizer.zero_grad()
        predictions = model(user_batch, item_batch).squeeze()
        loss = criterion(predictions, rating_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(dataloader)
    print(f'Epoch {epoch + 1}/{epochs}, Average Loss: {average_loss:.4f}')

Training Collaborative Model:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/1:   0%|          | 0/38095 [00:00<?, ?it/s]

Epoch 1/1, Average Loss: 1.3706


In [21]:
user_index = torch.tensor([5], dtype=torch.long).to(device)
item_index = torch.tensor([1], dtype=torch.long).to(device)
prediction = model(user_index, item_index).item()
print(f'Predicted Rating: {prediction:.4f}')

Predicted Rating: 4.9169


In [22]:
user_index = torch.tensor([user_to_index['A30TK6U7DNS82R']], dtype=torch.long)
item_index = torch.tensor([item_to_index['The Insiders (Insiders (Bloomsbury))']], dtype=torch.long)
user_index , item_index= user_index.to(device), item_index.to(device)
prediction = model(user_index, item_index).item()
print(f'Predicted Rating: {prediction:.4f}')

Predicted Rating: 4.2094


In [23]:
# Save the collaborative filtering model
model_path = 'collaborative_filtering_model.pth'
# Save the entire model
torch.save(model, model_path)
print(f'Model saved to {model_path}')

Model saved to collaborative_filtering_model.pth
